In [1]:
import tensorflow as tf
import tensorflow.keras as keras 
from tensorflow.keras import layers

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def plot_graph(history):
    plt.plot(history.epoch, history.history['loss'], label='loss')
    plt.plot(history.epoch, history.history['accuracy'], label='accuracy')
    plt.plot(history.epoch, history.history['val_accuracy'], label='val accuracy')
    plt.ylabel('%')
    plt.xlabel('epoch')
    plt.legend()
    plt.show()

In [ ]:
LEARNING_RATE = 0.02
to_be_resized = [
    'Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology',
    'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
    'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
    'Horizontal_Distance_To_Fire_Points'
]

In [ ]:
df = pd.read_csv('cover_data.csv')
data = df.iloc[:,0:-1]
labels = df.iloc[:,-1:]

train_dt, test_dt, train_labels, test_labels = train_test_split(data, labels, test_size=0.2, random_state=32)


std = ColumnTransformer([('standardize', StandardScaler(), to_be_resized)], remainder='passthrough')

In [ ]:
scaled_train_dt = pd.DataFrame(std.fit_transform(train_dt))
scaled_test_dt = pd.DataFrame(std.transform(test_dt))
scaled_train_lb = train_labels - 1
scaled_test_lb = test_labels - 1

In [ ]:
def create_model(l_rate, data):
    model = keras.Sequential()
    # Input
    model.add(layers.InputLayer(input_shape=(data.shape[1],)))
    # Hidden layers
    model.add(layers.Dense(128, activation='relu'))
    # model.add(layers.Dropout(0.25))
    # Hidden layers
    model.add(layers.Dense(32, activation='relu'))
    # model.add(layers.Dropout(0.1))
    # Hidden layers
    # model.add(layers.Dense(14, activation='relu'))
    # Output
    model.add(layers.Dense(7, activation='softmax'))
    # Compile
    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer='adam',
        metrics=['accuracy']
    )
    # keras.optimizers.Adam(learning_rate=l_rate)
    return model

In [ ]:
stop = keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0.0001, patience=5)

model = create_model(LEARNING_RATE, train_dt)
model.summary()

In [ ]:
history = model.fit(scaled_train_dt, scaled_train_lb, batch_size=1024, epochs=100, validation_split=0.1, callbacks=[stop])

plot_graph(history)

In [ ]:
y_pred = model.predict(scaled_test_dt)

# Convert the pred to discrete values
y_pred = np.argmax(y_pred, axis=1)
class_names = ['Spruce/Fir', 'Lodgepole Pine',
                'Ponderosa Pine', 'Cottonwood/Willow',
                'Aspen', 'Douglas-fir', 'Krummholz']
print(classification_report(scaled_test_lb, y_pred, target_names=class_names))